# Word Embedding 이란

- **워드 임베딩(word embedding)** 은 단어를 기호나 정수(ID) 대신, **의미 정보를 담은 실수 기반의 밀집 벡터(dense vector)** 로 표현하는 기술이다.    
즉, 단어를 고정 길이의 실수 벡터로 매핑하여 컴퓨터가 **단어 의미의 유사성**을 계산할 수 있게 한다. 이때 만들어진 벡터를 **임베딩 벡터(embedding vector)** 라고 부른다.
    - 의미의 유사성이 반영된다는 의미: **비슷한 의미를 가지는 단어는 비슷한 값들로 구성**되도록 하는 것을 말한다.
- 자연어 처리 모델은 단어 자체를 이해하지 못하므로, 단어를 숫자로 바꾸어야 한다. 하지만 단순히 숫자로 변환하는 방식(one-hot 등)은 단어 의미를 담지 못한다. 워드 임베딩은 이러한 문제를 해결하기 위해 등장했다.

### 희소(sparse) 표현
- 단어를 표현하는 대표적인 희소표현은 one-hot encoding이 있다. 
- 모든 단어를 포함하는 Vector의 차원에서 index 역할을 하는 하나의 숫자만 1이며 나머지는 모두 0으로 표현한다. 
- 단어개수가 vector의 차원이 되므로 단어가 많아 질 수록 벡터의 길이가 선형적으로 증가한다. 
- One-hot vector는 각 단어를 다르게 표현할 뿐 각 단어의 의미를 표현하지 못한다.

| 단어      | 단어 사전 크기 | 벡터 형태                   | 특징              |
| ------- | -------- | ----------------------- | --------------- |
| 예: “먹다” | 10,000   | `[0,0,0,0,...,1,...,0]` | 10,000차원 중 한 위치만 1, 나머지는 0 |


### 밀집(dense) 표현
- 밀집 표현은 단어를 고정된 차원(ex: 50, 100, 700 차원)의 실수 벡터로 표현한다.   
- 벡트의 길이가 작고 효율적이다. 
- 각 단어 벡터들은 단어의 의미를 가지는 실수들로 구성된다.
- 단어의 의미의 유사성이 벡터의 방향, 거리로 표현되어 단어간 유사성을 계산할 수있다. 

## 분산표현(Distributed Representation)

- 분포가설을 기반으로 하는 이론

### 기본 개념: 분포 가설(Distributional Hypothesis)

언어학에서 오래전부터 사용된 원리로, 다음과 같은 내용이다.
> “비슷한 맥락(context)에 등장하는 단어는 비슷한 의미를 가진다.”

즉, **단어의 의미는 그 주변 단어들의 분포로부터 파악할 수 있다.**

### 예:

- 나는 **밥을** 먹는다.
- 나는 **빵을** 먹는다.
- 나는 **국수를** 먹는다.

세 문장에서 
‘나는’과 ‘먹는다’ 사이에 **밥 / 빵 / 국수**가 각각 등장한다.

따라서

- 주변 단어(맥락)가 유사하므로
- **밥–빵–국수는 의미적으로 비슷한 음식 범주**로 학습된다.

이 원리를 이용해 단어 의미를 벡터로 학습하는 것을 **분산표현(Distributed Representation)** 이라고 한다.

- **word2vec, fastext** 등이 **분산표현**을 기반으로 만들어진 대표적인 모델이다.

# Word2Vec
- 딥러닝 기반 word embedding 방식. 딥러닝 모델은 `입력층-은닉층-출력층` 의 단순한 구조로 되어있다.
    - Fully connected layer로 이뤄진 단순한 딥러닝 모델이다.
    - 학습이 끝나면 Fully connected layer의 weights 를 embedding vector로 사용한다.
  
## CBOW, Skip-gram
- Word2Vec 학습 방식은 **CBOW(Continuous Bag of Words)와 Skip-gram 방식** 으로 나뉜다.    
- 두 방식 모두 분포가설에 입각해 주변단어와 중심단어간의 관계를 학습하여 embedding vector를 만든다. 
- CBOW 모델은 주변 단어들로부터 중심 단어를 예측하는 방식이다.
    -  나는 어제 ___ 함께 학교에 갔다.
- Skip-gram 모델은 중심 단어로부터 주변 단어들을 예측하는 방식이다.
    -   ____ 밥을 _____
 
![cbow_skipgram_outline](figures/word2vec_cbow_skipgram_outline.png)
> 중심단어: **target 단어**, 기준 단어<br>
> 주변단어: **target 주위**의 단어.  **context(맥락), window**라고 한다.<br>
> 주변단어의 크기를 몇개로 할지를 **window size** 라고 한다. window size를 N으로 하면 중심단어 **좌우로 N개**가 주변단어가 된다. <br>
> 

### CBOW (Continuous Bag of Words)
- CBOW 모델은 주변 단어들로부터 중심 단어를 예측하는 방식

#### Dataset 구성
- CBOW/Skip-gram 모델 학습 데이터셋은 모두 token들이 one-hot encoding 되 있어야 한다.
  - **one hot vector**
    
    <img src="figures/wordwvec_dataset_onehotvector.png">
- Window size를 설정 한 뒤 문장을 중심 단어를 뒤로 이동 시키면서 중심단어와 주변단어들을 추출해 데이터셋을 만든다.
    > - 지정한 개수(window) 만큼씩 이동하면서 어떤 작업을 진행하는 것을 **Sliding window 방식** 이라고 한다.
- <span style="font-size:1.2em;font-weight:bold">window size 를 2로 설정한 뒤 데이터셋 구성 예</span>
    - **sliding window**
      
       <img src="figures/word2vec_cbow_sliding_window.png" width="700">


#### 모델 구조
**주변단어들의 one-hot vector를 Input, 중심단어의 one-hot vector를 output**으로 하여 CBOW 모델을 학습한다.   
- 예)
  - **vocab size(총단어수)**: 10,000
  - **window size**: 2
  - **embedding 차원**: 100
  - **입력**: 주변단어(one-hot vector) - 4 (window_size x 2)
  - **출력**: 전체 단어들에 대해 중심 단어일 확률 출력(**다중분류 문제**)
   
<img src="figures/word2vec_cbow.png" width="800">

- 입력 단어는 **one-hot vector** 다. 그래서 단어 index만 1이고 나머진 모두 0으로 구성된다.
이  입력 one-hot vector와 가중치 행렬 $W_{in}$이 가중합(행렬곱)을 계산하면 가중치 행렬에서 그 단어 index의 행(one-hot vector의 1의 index의 행)  행을 가져오는 것이 된다. 그래서 word2vec의 hidden layer를 계산하는 작업($X \cdot W_{in}$)은 **가중치 행렬 $W_{in}$에서 해당 단어에 해당하는 행을 찾는(lookup) 작업**을 하는 것이 된다.
    - ex)
\begin{align} 
\left[
\begin{matrix}
    0 & 0& 1 & 0
\end{matrix}
\right] \cdot \left[
\begin{matrix}
0.1 & 0.1 & 0.1 \\
0.2 & 0.2 & 0.2 \\
0.3 & 0.3 & 0.3 \\
0.4 & 0.4 & 0.4
\end{matrix}
\right] = \left[
\begin{matrix}
0.3 & 0.3 & 0.3
\end{matrix}
\right] \\
\text{행: 단어(4), 열: embedding 차원(3)}
\end{align}    
- Word2Vec의 학습은 **가중치 행렬의 각 행들이 단어들의 word embedding vector**가 되도록 **주변단어와 중심단어의 관계로 학습**하는 것이다.
- 학습이 완료 되면 $W_{in}$ 이나 $W_{out}$ 파라미터를 word embeding vector로 사용한다.




### Skip-gram

중심단어를 이용해 그 주변단어를 예측하는 모델을 구성한다.

#### Dataset 구성
- CBOW 모델과 마찬가지로 모든 token 들은 one-hot vector로 구성한다.
- **sliding window** 방식으로 이동하면서 **중심단어와 그 주변단어 쌍**으로 Dataset을 구성한다.
    - window size를 n으로 지정하면 중심단어를 input 그 중심단의 앞의 n개, 뒤의 n가 단어를 output으로 구성한다.
- <span style="font-size:1.2em;font-weight:bold">window size 를 2로 설정한 뒤 데이터셋 구성 예</span> 

  <img src="figures/wore2vec_skipgram_sliding_window.png" width="700">

#### 모델 구조
<img src='figures/word2vec_skipgram.png' width="800">

- 모델의 입력으로 중심단어의 one-hot vector가 들어가고 모델을 주변단어를 추론한다. window size 가 2라면 총 4개의 주변단어를 추론한다.
- 학습 할 때는 각 주변단어들에 대한 개별 loss를 계산하고 그 합계를 최종 loss하여 $W_{in}$과 $W_{out}$ 을 update 한다.

### CBOW vs Skip-gram

- 여러 논문들에서 두 방식에 대한 성능 비교를 한 결과를 보면 대체적으로 **Skip-gram** 이 더 좋다.
    - 특히 말뭉치(Corpus)가 클수록 저빈도 단어나 유추 문제의 성능이 skip-gram이 더 뛰어난 경향이 있다.
    - skip-gram은 하나의 단어를 이용해 그 주변단어 여러개를 예측하는 것으로 주변단어로 중심단어 하나를 찾는 것(CBOW)보다 어려운 문제이다. Skip-gram이 CBOW 보다 더 어려운 상황에서 훈련하는 모델인 만큼 단어의 분산표현이 더 뛰어날 가능성이 커진다.

# Fastext
- Word2Vec 기반의 word embedding 방식. 모델 구조는 word2vec과 동일하다.
- Word2Vec은 단어(word) 단위로 학습을 하는데 비해 **Fastext**는 단어를 subword로 더 나눠 학습한다.

## 장점
- Subword 방식으로 단어를 더 쪼개서 학습하기 때문에 OOV(Out of Vocabulary) 에 대해 강점을 가진다.